In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import matplotlib.dates as dts
import numpy as np
import pandas as pd
import itertools
import os
import ROOT
import datetime
from root_numpy import root2array, root2rec, tree2rec, array2root
from scipy.optimize import curve_fit
from scipy.misc import factorial
plt.rcParams.update({'font.size': 16})

In [ ]:
from larlite import larlite

In [ ]:
fin = ROOT.TFile('/home/david/uboone/data/mcc7/cosmics/photonclusters.root')
t_clus = fin.Get('cluster_photon_tree')
t_hit  = fin.Get('hit_gaushit_tree') 

In [ ]:
t2cm = 0.05571
w2cm = 0.3

In [ ]:
def getHitVector(n):
    t_hit.GetEntry(n)
    ev_hit_v = t_hit.hit_gaushit_branch
    hit_w_v = [ x.WireID().Wire * w2cm for x in ev_hit_v if (x.WireID().Plane == 2)]
    hit_t_v = [ x.PeakTime()    * t2cm for x in ev_hit_v if (x.WireID().Plane == 2)]
    hit_q_v = [ x.Integral()           for x in ev_hit_v if (x.WireID().Plane == 2)]
    return hit_w_v, hit_t_v, hit_q_v

In [ ]:
Zmax = 1036
Xmax = 360
boxwidth = 50

Z = np.arange(boxwidth/2., Zmax-boxwidth/2., boxwidth)
X = np.arange(boxwidth/2., Xmax-boxwidth/2., boxwidth)
Z, X = np.meshgrid(Z, X)

Nz = Zmax/boxwidth + 1
Nx = Xmax/boxwidth + 1

ZmaxBox = Nz * boxwidth
XmaxBox = Nx * boxwidth

def GenerateHeatMap(n):
    t_clus.GetEntry(n)
    clus_v = t_clus.cluster_photon_branch
    Y_clus_v = [clus for clus in clus_v if ( (clus.View() == 2) and (clus.NHits() < 30) and (clus.NHits() > 2) )]
    print 'there are %i clusters'%len(Y_clus_v)
    # create list of average cluster locations
    Y_clus_loc_v = []
    for clus in Y_clus_v:
        w_avg = 0.5 * (clus.StartWire() + clus.EndWire())
        t_avg = 0.5 * (clus.StartTick() + clus.EndTick())
        Y_clus_loc_v.append( (int(w_avg * w2cm), int(t_avg * t2cm)) )
        
    #print Y_clus_loc_v
    
    mat = [ [ 0 for x in range(Nz)] for y in range(Nx)]
    matpos = [ [ (0,0) for x in range(Nz)] for y in range(Nx)]
    
    for z in xrange(Nz):
        for x in xrange(Nx):
            # count how many clusters in this block
            zpos = boxwidth * (z + 0.5)
            xpos = boxwidth * (x + 0.5)
            matpos[x][z] = (zpos,xpos)
            #print 'box position : [%i, %i]'%(int(zpos),int(xpos))
            
            for clus_loc in Y_clus_loc_v:
                #print '\t cluster position : [%i, %i]'%(clus_loc[0],clus_loc[1])
                d = np.sqrt( (clus_loc[0] - zpos)**2 + (clus_loc[1] - xpos)**2 )
                #print '\t distance : ',d
                if (d < boxwidth):
                    mat[Nx-x-1][z] += 1
    # split the TPC in a 2D map of 50 cm blocks
    #print matpos
    return mat
    


In [ ]:
for n in xrange(100):
    
    hit_w, hit_t, hit_q_v = getHitVector(n)
    mat = GenerateHeatMap(n)
    
    npmat = np.array(mat)
    
    maxval = 0
    for i in mat:
        for j in i:
            if j > maxval: maxval = j
    if (maxval < 7): continue

    fig = plt.figure(figsize=(17,8))

    EXTENT = 0, ZmaxBox, 0, XmaxBox
    plt.imshow(mat,extent=EXTENT,interpolation='nearest',alpha=0.5, vmin=0, vmax=10)

    #sc = plt.scatter(hit_w,hit_t,s=20,c=hit_q_v , vmin=0, vmax=400, edgecolors='')
    plt.scatter(hit_w,hit_t,s=20, facecolors='k', edgecolors='')
    #plt.colorbar(sc)
    plt.grid()
    plt.xlim(-10,ZmaxBox+10)
    plt.title('Entry %i'%n)

    plt.show()

In [ ]:
# show cluster integral vs. size:

clus_integral_v = []
clus_size_v     = []

for n in xrange(100):
    t_clus.GetEntry(n)
    clus_v = t_clus.cluster_rawcluster_branch
    Y_clus_v = [clus for clus in clus_v if ( (clus.View() == 2) and (clus.NHits() > 0) )]
    for clus in Y_clus_v:
        clus_integral_v.append( clus.Integral())
        clus_size_v.append( clus.NHits() )

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.hist(clus_integral_v,bins=np.linspace(0,200,100))
plt.yscale('log')
plt.grid()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))

BINS = (np.linspace(0,500,100),np.linspace(0,10,11))
from matplotlib.colors import LogNorm

plt.hist2d(clus_integral_v,clus_size_v,bins=BINS,norm=LogNorm())
#plt.scatter(clus_integral_v,clus_size_v)#,bins=BINS)
#plt.xlim([0,2000])
#plt.ylim([0,20])
plt.grid()
plt.show()